In [2]:
#! pip install keras pandas efficientnet sklearn scikit-image opencv-python
#! apt-get install -y libsm6 libxext6 libxrender-dev
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from efficientnet import EfficientNetB3, center_crop_and_resize, preprocess_input

from keras import models
from keras import layers
from keras import callbacks

import time

import matplotlib.pyplot as plt

import cv2

In [3]:
BASEPATH = "stanford-dogs-dataset/"
BATCH_SIZE = 32
EPOCHS = 100

In [4]:
early_stopping = EarlyStopping(patience=10, verbose=1,restore_best_weights=True, monitor="val_acc")
reduce_lr = callbacks.ReduceLROnPlateau(factor=0.1, patience=3,verbose=1, monitor="val_acc")

In [16]:
def build_model(conv_base, trainable=False, dense_layers=2):
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.GlobalAveragePooling2D())
    for _ in range(dense_layers):
        model.add(layers.Dense(2048))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))
    model.add(layers.Dense(512))
    #model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(120,activation='softmax'))
    
    if not trainable:
        for layer in conv_base.layers:
            layer.trainable = False

    model.compile("adam",loss="categorical_crossentropy",metrics=["accuracy"])

    print(model.summary())
    
    return model

In [17]:
conv_base=EfficientNetB3(weights='imagenet',include_top=False, input_shape=(224,224,3))

model = build_model(conv_base,dense_layers=0)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b3 (Model)      (None, 7, 7, 1536)        10783528  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1536)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               786944    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 120)               61560     
Total params: 11,632,032
Trainable params: 848,504
Non-trainable params: 10,783,528
_________________________________________________________________
None


# Using the ImageDataGenerator

In [18]:
train_datagen = ImageDataGenerator(rescale=1./255.0,
                                   shear_range=0.1,
                                   rotation_range=10.,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=[0.9, 1.25],
                                   brightness_range=[0.8, 1.2],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255.0)

validation_datagen = ImageDataGenerator(rescale=1./255.0)

In [19]:
train_generator = train_datagen.flow_from_directory(
    BASEPATH + "train",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    BASEPATH + "test",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    BASEPATH + "val",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 15394 images belonging to 120 classes.
Found 1114 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [20]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = EPOCHS, callbacks=[early_stopping, reduce_lr])

Epoch 1/100
481/481 [==============================] - 384s 798ms/step - loss: 2.6635 - acc: 0.3683 - val_loss: 1.2165 - val_acc: 0.6631
Epoch 2/100
481/481 [==============================] - 364s 756ms/step - loss: 1.9597 - acc: 0.4940 - val_loss: 1.1655 - val_acc: 0.6733
Epoch 3/100
481/481 [==============================] - 362s 753ms/step - loss: 1.8855 - acc: 0.5097 - val_loss: 1.1466 - val_acc: 0.6822
Epoch 4/100
481/481 [==============================] - 362s 753ms/step - loss: 1.8424 - acc: 0.5226 - val_loss: 1.1659 - val_acc: 0.6760
Epoch 5/100
481/481 [==============================] - 362s 752ms/step - loss: 1.7916 - acc: 0.5355 - val_loss: 1.1612 - val_acc: 0.6832
Epoch 6/100
481/481 [==============================] - 364s 756ms/step - loss: 1.7570 - acc: 0.5468 - val_loss: 1.1685 - val_acc: 0.6809
Epoch 7/100
481/481 [==============================] - 363s 755ms/step - loss: 1.7523 - acc: 0.5467 - val_loss: 1.1522 - val_acc: 0.6891
Epoch 8/100
481/481 [====================

## Evaluation

In [23]:
loss, acc = model.evaluate_generator(test_generator,verbose=0, steps=test_generator.samples // BATCH_SIZE)

In [24]:
print(loss,acc)

1.0938490585369223 0.6939338235294118


In [32]:
model.save("efficientnetb3-model-{:.2f}acc-{:.2f}loss-{:.0f}.hdf5".format(acc, loss, time.time()))